# MET

## Loading the Data

In [ ]:
import numpy as np
np.random.seed(42)
%matplotlib inline

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import scipy 

In [ ]:
#Loading the data: returning data and filters for stimulus and baseline for both muscles
def get_data(base=1, method='subtraction', zstand=1, resolution=100, outl='sd', exclude='no'):
   
    #Function for Outlier-Detection
    def outlier_trials(df, outl):
        for i in set(df.vpn.values):
            
            target=[zyg_s, cor_s, scl_s]

            for t in target:
                
                werte=np.array(df.loc[(df.vpn==i), t]) 
                         
                trial_mean=np.nanmean(werte, axis=1)
                vpn_mean=np.nanmean(trial_mean, axis=0)            
                vpn_std=np.nanstd(trial_mean, axis=0)

                if outl=='sd':
                    out_ind=(np.abs(trial_mean-vpn_mean))>3*vpn_std
                    
                if outl=='iq':
                    quartile_1, quartile_3 = np.percentile(trial_mean, [25, 75])
                    iqr = quartile_3 - quartile_1
                    lower_bound = quartile_1 - (iqr * 3)
                    upper_bound = quartile_3 + (iqr * 3)
                    out_ind=(trial_mean > upper_bound) | (trial_mean < lower_bound)
                werte[out_ind]=np.nan   
                df.loc[(df.vpn==i), t]=werte
               
        return df

    def outlier(df, outl):
        target=[zyg_s, zyg_b, cor_b, cor_s, scl_b, scl_s]
        for i in set(df.vpn.values):
            
            for t in target:
                werte=np.array(df.loc[(df.vpn==i), t])
                rows=len(werte)
                columns=len(werte.T)
                werte_r=werte.reshape(1,(rows*columns))
                
                if outl=='iq':
                    quartile_1, quartile_3 = np.percentile(werte_r, [25, 75])
                    iqr = quartile_3 - quartile_1
                    lower_bound = quartile_1 - (iqr * 1.5)
                    upper_bound = quartile_3 + (iqr * 1.5)
                    werte_r[np.where((werte_r > upper_bound) | (werte_r < lower_bound))]=np.nan
                if outl=='sd':
                    multi=3
                    werte_r[np.abs(werte_r-np.nanmean(werte_r))>multi*np.nanstd(werte_r)]=np.nan
                
                werte_o=werte_r    
                werte_new=werte_o.reshape(rows,columns)  
                df.loc[(df.vpn==i), t]=werte_new  
        return df
    
    #Function for Z-Standardisation           
    def mimstand(df):
        vpn=df.vpn.unique()
            
        for i in vpn:
            target=[zyg_s, cor_s, scl_s] 
            for t in target:
                orig=df.loc[(df.vpn==i),t]
                orig_mu=orig.mean().mean()
                orig_stand=orig.std().mean()
                zdata=(orig-orig_mu)/orig_stand
                df.loc[(df.vpn==i),t]=zdata 
        return df
    
    # Loading of the Datafile (3 possible resolutions and naming of the columns)
    if resolution==50:
        filename='MET50_af.csv'
        df=pd.read_csv(filename,sep=',', na_values=['?'])   
        #Baseline = 0-1000ms
        scl_b=df.loc[:,'METSCL1':'METSCL19'].columns
        #Stimulus = 500ms-4000ms after trigger
        scl_s=df.loc[:,'METSCL40':'METSCL110'].columns
        
    if (resolution==500):
        filename='MET500_af.csv' 
        df=pd.read_csv(filename,sep=',', na_values=['?'])
             
        if base==1:
            print 'base: 0ms-500ms'
            zyg_b=df.loc[:,'METZYG1':'METZYG1'].columns       
            cor_b=df.loc[:,'METCOR1':'METCOR1'].columns
            scl_b=df.loc[:,'METSCL1':'METSCL1'].columns
        if base==2:
            print 'base: 0ms-1000ms'
            zyg_b=df.loc[:,'METZYG1':'METZYG2'].columns
            cor_b=df.loc[:,'METCOR1':'METCOR2'].columns
            scl_b=df.loc[:,'METSCL1':'METSCL2'].columns  
        if base==3:
            print 'base 500ms-1000ms'
            zyg_b=df.loc[:,'METZYG2':'METZYG2'].columns
            cor_b=df.loc[:,'METCOR2':'METCOR2'].columns         
            scl_b=df.loc[:,'METSCL2':'METSCL2'].columns
            
        zyg_s=df.loc[:,'METZYG5':'METZYG11'].columns
        cor_s=df.loc[:,'METCOR5':'METCOR11'].columns
        scl_s=df.loc[:,'METSCL5':'METSCL11'].columns
        
    if (resolution==100):
        filename='MET100_af.csv'     
        df=pd.read_csv(filename,sep=',', na_values=['?'])
             
        if base==1:
            print 'base: 0ms-500ms'
            zyg_b=df.loc[:,'METZYG1':'METZYG5'].columns       
            cor_b=df.loc[:,'METCOR1':'METCOR5'].columns
            scl_b=df.loc[:,'METSCL1':'METSCL5'].columns
        if base==2:
            print 'base: 0ms-1000ms'
            zyg_b=df.loc[:,'METZYG1':'METZYG9'].columns
            cor_b=df.loc[:,'METCOR1':'METCOR9'].columns
            scl_b=df.loc[:,'METSCL1':'METSCL9'].columns
        if base==3:
            print 'base 500ms-1000ms'
            zyg_b=df.loc[:,'METZYG5':'METZYG9'].columns
            cor_b=df.loc[:,'METCOR5':'METCOR9'].columns
            scl_b=df.loc[:,'METSCL5':'METSCL9'].columns
            
        scl_s=df.loc[:,'METSCL20':'METSCL55'].columns
        zyg_s=df.loc[:,'METZYG20':'METZYG55'].columns
        cor_s=df.loc[:,'METCOR20':'METCOR55'].columns   
    
    #exclude weird participant
    if exclude=='yes':
        df=df[df.vpn!=38].reset_index(drop=True)
        df=df[df.vpn!=22].reset_index(drop=True)
        df=df[df.vpn!=31].reset_index(drop=True)
        df=df[df.vpn!=132].reset_index(drop=True)

        
    target=[zyg_s, zyg_b, cor_b, cor_s, scl_b, scl_s]
    for t in target:
        #NaN setzen von Trials, bei denen mehr als 25% NaNs
        numberNaNs=df[t].isnull().sum(axis=1)
        df.loc[(numberNaNs>(len(t)*0.30)), t]=np.nan

    
    #Baseline-Correction
    # raw data
    if method=='raw':
        print 'raw data'
    
    #1500ms baseline
    if method=='subtraction': 
        print 'baseline subtraction'
        df[zyg_s]=df[zyg_s].sub(np.nanmean(df[zyg_b], axis=1), axis=0)
        df[cor_s]=df[cor_s].sub(np.nanmean(df[cor_b], axis=1), axis=0)
        
        df[zyg_b]=df[zyg_b].sub(np.nanmean(df[zyg_b], axis=1), axis=0)
        df[cor_b]=df[cor_b].sub(np.nanmean(df[cor_b], axis=1), axis=0)
        
        df[scl_s]=df[scl_s].sub(np.nanmean(df[scl_b], axis=1), axis=0)
        
 
    #Division durch Baseline
    if method=='division':
        print 'baseline division'
        df[zyg_s]=df[zyg_s].divide(np.nanmean(df[zyg_b], axis=1), axis=0)
        df[cor_s]=df[cor_s].divide(np.nanmean(df[cor_b], axis=1), axis=0)
        
        df[zyg_b]=df[zyg_b].divide(np.nanmean(df[zyg_b], axis=1), axis=0)
        df[cor_b]=df[cor_b].divide(np.nanmean(df[cor_b], axis=1), axis=0)

        df[scl_s]=df[scl_s].divide(np.nanmean(df[scl_b], axis=1), axis=0)


    #Division durch Trial
    if method=='trial_division':
        print 'trial division'
        df[zyg_s]=df[zyg_s].divide(np.nanmean(df[zyg_s], axis=1), axis=0)
        df[cor_s]=df[cor_s].divide(np.nanmean(df[cor_s], axis=1), axis=0)
        
        df[zyg_b]=df[zyg_b].divide(np.nanmean(df[zyg_b], axis=1), axis=0)
        df[cor_b]=df[cor_b].divide(np.nanmean(df[cor_b], axis=1), axis=0)
        
        df[scl_s]=df[scl_s].divide(np.nanmean(df[scl_s], axis=1), axis=0)

    #Trial-Outlier-Detection
    if outl=='orig':
        print 'no outlier detection'
    else:
        df=outlier_trials(df, outl)
        
    #Z-standardisation       
    if zstand==1:
        df=mimstand(df)
    
    #make columns easier accesible with labels 
    fragebogen=['VPN', 'SPF_SUM', 'SPF_FS_SUM', 'SPF_EC_SUM', 'SPF_PT_SUM',
       'SPF_PD_SUM', 'SPF_Empathy_SUM', 'AQsum', 'NPIgesamt', 'SOCsum',
       'PANAS01pos', 'PANAS01neg', 'PANAS02pos', 'PANAS02neg']
    MET_num=['key', 'time']
    MET_cat=['con', 'cor', 'gen', 'age', 'val']
    
    df['all']=np.ones(len(df))
    df['gender']=df.vpn>100
    
    #Exlude cases, where a participant has not answered a question
    #df[df[fragebogen]==0]=np.nan
    
    #Name the triggers with conditions
    df.loc[(df.triggers==101), 'triggers']='pos_cog'
    df.loc[(df.triggers==102), 'triggers']='neg_emo'
    df.loc[(df.triggers==103), 'triggers']='pos_emo'
    df.loc[(df.triggers==104), 'triggers']='neg_cog'
    df['all']=np.ones(len(df))
     
    return df, zyg_b, cor_b, zyg_s, cor_s, scl_s, scl_b, fragebogen, MET_num, MET_cat

In [ ]:
def prepare(df, zyg_s, cor_s, scl_s):
    
    ## Physiological Measure
    df['zyg_mean']=df[zyg_s].mean(axis=1)
    df['cor_mean']=df[cor_s].mean(axis=1)
    df['scl_mean']=df[scl_s].mean(axis=1)

    df['zyg_max']=df[zyg_s].max(axis=1)
    df['cor_max']=df[cor_s].max(axis=1)
    df['scl_max']=df[scl_s].max(axis=1)

    df['zyg_min']=df[zyg_s].min(axis=1)
    df['cor_min']=df[cor_s].min(axis=1)
    df['scl_min']=df[scl_s].min(axis=1)

    df['zyg_var']=df[zyg_s].var(axis=1)
    df['cor_var']=df[cor_s].var(axis=1)
    df['scl_var']=df[cor_s].var(axis=1)

    df['zyg_skew']=df[zyg_s].skew(axis=1)
    df['cor_skew']=df[cor_s].skew(axis=1)
    df['scl_skew']=df[cor_s].skew(axis=1)

    df['zyg_kurt']=df[zyg_s].kurtosis(axis=1)
    df['cor_kurt']=df[cor_s].kurtosis(axis=1)
    df['scl_kurt']=df[scl_s].kurtosis(axis=1)

    #Mimicry-Measures
    neg_mim= np.nanmean((np.array(df[cor_s][df['val']=='neg'])-np.array(df[zyg_s][df['val']=='neg'])), dtype=np.float64, axis=1)
    pos_mim= np.nanmean(np.array(df[zyg_s][df['val']=='pos'])-np.array(df[cor_s][df['val']=='pos']), dtype=np.float64, axis=1)
    df['mim_mean']=np.zeros(len(df))
    df.loc[(df['val']=='neg'), 'mim_mean']=neg_mim
    df.loc[(df['val']=='pos'), 'mim_mean']=pos_mim

    neg_mimMAX= np.max((np.array(df[cor_s][df['val']=='neg'])-np.array(df[zyg_s][df['val']=='neg'])), axis=1)
    pos_mimMAX= np.max(np.array(df[zyg_s][df['val']=='pos'])-np.array(df[cor_s][df['val']=='pos']), axis=1)
    df['mim_max']=np.zeros(len(df))
    df.loc[(df['val']=='neg'), 'mim_max']=neg_mimMAX
    df.loc[(df['val']=='pos'), 'mim_max']=pos_mimMAX

    #Gender
    df['female']=df.VPN>100

    #Creating handcrafted Features
    handcrafted=['zyg_mean', 'cor_mean', 'zyg_max', 'cor_max', 'zyg_min', 'cor_min', 
                          'zyg_var', 'cor_var', 'zyg_skew', 'cor_skew', 'zyg_kurt', 'cor_kurt']

    ## Participants traits  
    df['coge_vpn']=df.loc[(df.con==1),'cor'].groupby(df['vpn']).transform('mean')
    df['emoe_vpn']=df.loc[(df.con==2),'key'].groupby(df['vpn']).transform('mean')
    beh=['coge_vpn', 'emoe_vpn']
    
    ## Pic traits ## Participants traits  
    df['coge_pic']=df.loc[(df.con==1),'cor'].groupby(df['pic']).transform('mean')
    df['emoe_pic']=df.loc[(df.con==2),'key'].groupby(df['pic']).transform('mean')
    df['rt_pic']=df.loc[(df.con==1),'time'].groupby(df['pic']).transform('mean')   
    
    #recode valence and condition labels
    df.loc[(df['val']=='pos'), 'val']=1
    df.loc[(df['val']=='neg'), 'val']=0
    df.loc[(df['con']==2), 'con']=0
    df['val']=pd.to_numeric(df.val)

    return df, handcrafted

In [ ]:
df, zyg_b, cor_b, zyg_s, cor_s, scl_s, scl_b, fragebogen, MET_num, MET_cat=get_data(base=2, 
                                                                                 zstand=1, 
                                                                                 method='subtraction', 
                                                                                 resolution=100,
                                                                                 outl='sd', 
                                                                                 exclude='yes')

df, handcrafted =prepare(df, zyg_s, cor_s, scl_s)

#df=df.fillna(method='ffill') #exclude NaNs
df=df.dropna(subset=zyg_s | cor_s)

In [ ]:
print len(df.groupby('vpn'))

## Regression (Function for Linear Regression and SVM)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn import svm
from sklearn.model_selection import ShuffleSplit

from sklearn import linear_model 
    
def regression(X, y, vpn=pd.DataFrame(), cv=3, hypercv=5):
    y_test = []
    y_mean=[]
    y_pred_lin = []
    y_pred_svm = []
    y_pred_tree = []    

    if vpn.empty:
        
        X=np.array(X)
        y=np.array(y)

        #shuffle the data 
        indices=np.arange(len(X))
        np.random.shuffle(indices)
        X=X[indices]
        y=y[indices]
        
        rs = ShuffleSplit(n_splits=3, test_size=.3)
        for train, test in rs.split(X):

            X_train=X[train]
            y_train=y[train]
        
            X_test=X[test]
   
            y_test = np.append(y_test, y[test])
            y_mean = np.append(y_mean, np.mean(y_train))   

            pred_lin_new, pred_svm_new=regress(X_train, X_test, y_train, y_test, hypercv)           
            y_pred_lin = np.append(y_pred_lin, pred_lin_new)
            y_pred_svm=np.append(y_pred_svm, pred_svm_new)
            
    
    else:

        vpns=np.array(vpn.unique())    

        rs = ShuffleSplit(n_splits=3, test_size=0.3)
        for train, test in rs.split(vpns):
        #train, test = train_test_split(vpns, test_size = 0.2)

            X_train=np.array(X[vpn.isin(vpns[train])])
            X_test=np.array(X[vpn.isin(vpns[test])])     
            
            y_train=np.array(y[vpn.isin(vpns[train])])
            y_test = np.append(y_test, np.array(y[vpn.isin(vpns[test])]))

            y_mean = np.append(y_mean, np.mean(y_train)) 
            
            pred_lin_new, pred_svm_new=regress(X_train, X_test, y_train, y_test, hypercv, do_svm=True)       
            
  
            y_pred_lin = np.append(y_pred_lin, pred_lin_new, axis=0)
            y_pred_svm = np.append(y_pred_svm, pred_svm_new, axis=0)
    
    acc_lr, acc_svm, acc_mean=reg_cross_eval(y_mean, y_test, y_pred_lin, y_pred_svm)    
    return acc_lr, acc_svm, acc_mean

In [ ]:
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

def regress(X_train, X_test, y_train, y_test, hypercv, do_svm=False):

    pred_svm = []
    
    #Festlegen der mit Crossvalidierung zu tunenden Parameter
    tuned_lin_parameters =[{'alpha':[1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4]}]

    tuned_svm_parameters = [{"C": [1e0, 1e-1, 1e-2, 1e-3],
                            "gamma": [1e-2, 1e-1, 1, 1e1, 1e2], #only for rbf
                            'degree':[1,2,3]}] #only for poly
    lin_clf=GridSearchCV(linear_model.Ridge(random_state=42), 
                         tuned_lin_parameters, cv=hypercv)

    lin_clf.fit(X_train, y_train)       
    pred_lin = lin_clf.predict(X_test)

    #Evaluation der Classifier auf den Training-Test-Splits
    print("Beste Parameter fuer Linear Regression auf einem der Trainingssets:")
    print(lin_clf.best_estimator_)
    
    svm_clf=GridSearchCV(SVR(kernel='rbf'), cv=hypercv, param_grid=tuned_svm_parameters) 

    svm_clf.fit(X_train, y_train)        
    pred_svm = svm_clf.predict(X_test)

    print("Beste Parameter fuer SVM auf einem der Trainingssets:")
    print(svm_clf.best_estimator_)

    return pred_lin, pred_svm;

In [ ]:
def reg_cross_eval(y_mean, y_test, y_pred_lin, y_pred_svm):
    
    #Evaluation der crossvalidierten Ergebnisse 
    print 'Crossvalidierte Ergebnisse'
    # The mean squared error
    acc_lr=np.sqrt(np.mean(((y_pred_lin) - y_test) ** 2))
    acc_svm=np.sqrt(np.mean(((y_pred_svm) - y_test) ** 2))
    acc_mean=np.sqrt(np.mean((np.mean((y_mean)) - y_test) ** 2))

    print("Root Mean Square Error der LR: %.2f"
        % np.sqrt(np.mean(((y_pred_lin) - y_test) ** 2)))

    #Explained variance score: 1 is perfect prediction
    #print('Variance score: %.2f' % lr.score(X_test, y_test))
    print('Korrelation: %.2f' % np.correlate(y_pred_lin, y_test))
    # The coefficients
    #print('Coefficients: \n', lr.coef_)
    
    print("Root Mean Square Error der SVM: %.2f"
          % np.sqrt(np.mean(((y_pred_svm) - y_test) ** 2)))
    
    print 'Prediction des Means'   
    print("Root Mean Square Error:  %.2f"
     % np.sqrt(np.mean((np.mean((y_mean)) - y_test) ** 2)))
    
    return acc_lr, acc_svm, acc_mean;

## Predictions with Linear Regression and SVR (itembased)

In [ ]:
print 'handcrafted'
#df=df.fillna(method='ffill')
emo_df=df[df['con']==0] #nur Items, bei denen Mitgefuehl angegeben werden musste
emo_df = emo_df.reset_index(drop=True) 

print 'Prediction of empathy with both Muscles (only stimulus-time)'
regression(emo_df[handcrafted], y=emo_df['key'], vpn=emo_df['vpn'])

print 'Prediction of empathy with both Muscles (only stimulus-time) - seperated by valence'
df_pic_pos=df.loc[((df.val==1) & (df.con==0)), :].reset_index(drop=True)
y=df_pic_pos.key
X=df_pic_pos[handcrafted]
regression(X, y, vpn=emo_df['vpn'])

df_pic_neg=df.loc[((df.val==0) & (df.con==0)), :].reset_index(drop=True)
y=df_pic_neg.key
X=df_pic_neg[handcrafted]
regression(X, y, vpn=emo_df['vpn'])

print 'mimicry'
X=pd.DataFrame(np.array(emo_df[cor_s])-np.array(emo_df[zyg_s]))
y=emo_df.key
regression(X, y, vpn=emo_df['vpn'])

In [ ]:
#ACHTUNG itembased ist es eigentlich ein Multiple-Class Classification-Task

emo_df=df[df['con']==0] #nur Items, bei denen Mitgefuehl angegeben werden musste
emo_df = emo_df.reset_index(drop=True) 

print 'Prediction of empathy with both Muscles (only stimulus-time)'
regression(pd.concat([emo_df[zyg_s], emo_df[cor_s]], axis=1), y=emo_df['key'], vpn=emo_df['vpn'])

print 'Prediction of empathy with both Muscles (only stimulus-time) - seperated by valence'
df_pic_pos=df[(df.val==1) & (df.con==0)].reset_index(drop=True) 
df_pic_neg=df[(df.val==0) & (df.con==0)].reset_index(drop=True) 
y=df_pic_pos.key
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s]], axis=1)
regression(X, y, vpn=emo_df['vpn'])

X=pd.concat([df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)
y=df_pic_neg.key
regression(X, y, vpn=emo_df['vpn'])

## PARTICIPANT OR PIC BASED PREDICTION!!

In [ ]:
print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic=df.loc[df.AQsum.notnull()].groupby(['vpn']).mean()
y=df_pic.AQsum
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
acc=regression(X, y)

print 'Prediction of SPF with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic=df.loc[df.SPF_SUM.notnull()].groupby(['vpn']).mean()
y=df_pic.SPF_SUM 
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
regression(X, y)

print 'Prediction of correct Anwsers with both Muscles - PARTICIPANTBASED'
pd.to_numeric(df.cor)
df_pic=df.loc[df.con==1].groupby(['vpn']).mean()
y=df_pic.cor
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
regression(X, y)

print 'Prediction of correct Anwsers with both Muscles - PICTUREBASED'
df_pic=df.loc[df.con==1].groupby(['pic']).mean()
y=df_pic.cor
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
regression(X, y)

print 'Prediction of Empathy with both Muscles - Participantbased'
df_pic=df.loc[df.con==0].groupby(['vpn']).mean()
y=df_pic.key
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
regression(X, y)

print 'Prediction of Empathy with both Muscles - PICTUREBASED'
df_pic=df.loc[df.con==0].groupby(['pic']).mean()
y=df_pic.key
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)
regression(X, y)

In [ ]:
print 'HANDCRAFTED'

print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic=df.loc[df.AQsum.notnull()].groupby(['vpn']).mean()
x1=df_pic[zyg_s]
x2=df_pic[cor_s]
y=df_pic.AQsum
X=df_pic[handcrafted]
regression(X, y)

print 'Prediction of SPF with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic=df.loc[df.SPF_SUM.notnull()].groupby(['vpn']).mean()
x1=df_pic[zyg_s]
x2=df_pic[cor_s]
y=df_pic.SPF_SUM 
X=df_pic[handcrafted]
regression(X, y)

print 'Prediction of correct Anwsers with both Muscles - PARTICIPANTBASED'
pd.to_numeric(df.cor)
df_pic=df.loc[df.con==1].groupby(['vpn']).mean()
y=df_pic.cor
X=df_pic[handcrafted]
regression(X, y)

print 'Prediction of correct Anwsers with both Muscles - PICTUREBASED'
df_pic=df.loc[df.con==1].groupby(['pic']).mean()
y=df_pic.cor
X=df_pic[handcrafted]
regression(X, y)

print 'Prediction of Empathy with both Muscles - Participantbased'
df_pic=df.loc[df.con==0].groupby(['vpn']).mean()
y=df_pic.key
X=df_pic[handcrafted]
regression(X, y)

print 'Prediction of Empathy with both Muscles - PICTUREBASED'
df_pic=df.loc[df.con==0].groupby(['pic']).mean()
y=df_pic.key
X=df_pic[handcrafted]
regression(X, y)


In [ ]:
## Prediction with more features for pos and neg Pictures

In [ ]:
print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic_pos=df[(df.val==1) & (df.AQsum.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.AQsum.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.AQsum
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s], df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)
regression(X, y)

#df.loc[(df[group1]==trigger)&(df[idx]==id)]                       
                        
print 'Prediction of SPF with both Muscles (only stimulus-time) - PARTICIPANTBASED'

df_pic_pos=df[(df.val==1) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.SPF_SUM 
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s], df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)
regression(X, y)

print 'Prediction of correct Anwsers with both Muscles - PARTICIPANTBASED'
pd.to_numeric(df_pic_pos.cor)
df_pic_pos=df[(df.val==1) & (df.con==1)].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.con==1)].groupby(['vpn']).mean()
y=df_pic_pos.cor
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s], df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)
regression(X, y)

print 'Prediction of Empathy with both Muscles - Participantbased'
df_pic_pos=df[(df.val==1) & (df.con==0)].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.con==0)].groupby(['vpn']).mean()
y=df_pic_pos.key
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s], df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)
regression(X, y)


In [ ]:
print 'Mimikry'
print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic_pos=df[(df.val==1) & (df.AQsum.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.AQsum.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.AQsum
X=np.hstack([(np.array(df_pic_pos[zyg_s])-np.array(df_pic_pos[cor_s])), 
             np.array(df_pic_neg[zyg_s])-np.array(df_pic_neg[cor_s])])

regression(X, y)

#df.loc[(df[group1]==trigger)&(df[idx]==id)]                       
                        
print 'Prediction of SPF with both Muscles (only stimulus-time) - PARTICIPANTBASED'

df_pic_pos=df[(df.val==1) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.SPF_SUM 
X=np.hstack([(np.array(df_pic_pos[zyg_s])-np.array(df_pic_pos[cor_s])), 
             np.array(df_pic_neg[zyg_s])-np.array(df_pic_neg[cor_s])])
regression(X, y)


print 'Prediction of Empathy with both Muscles - Participantbased'
df_pic_pos=df[(df.val==1) & (df.con==0)].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.con==0)].groupby(['vpn']).mean()
y=df_pic_pos.key
X=np.hstack([(np.array(df_pic_pos[zyg_s])-np.array(df_pic_pos[cor_s])), 
             np.array(df_pic_neg[zyg_s])-np.array(df_pic_neg[cor_s])])
regression(X, y)

In [ ]:
print 'HANDCRAFTED'
print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic_pos=df[(df.val==1) & (df.AQsum.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.AQsum.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.AQsum
#X=np.hstack([df_pic_pos[handcrafted], df_pic_neg[handcrafted]])
#regression(X, y)

#df.loc[(df[group1]==trigger)&(df[idx]==id)]                       
                        
print 'Prediction of SPF with both Muscles (only stimulus-time) - PARTICIPANTBASED'

df_pic_pos=df[(df.val==1) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.SPF_Empathy_SUM.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.SPF_SUM 
X=pd.concat([df_pic_pos[handcrafted], df_pic_neg[handcrafted]], axis=1)
regression(X, y)


print 'Prediction of correct Anwsers with both Muscles - PARTICIPANTBASED'
pd.to_numeric(df_pic_pos.cor)
df_pic_pos=df[(df.val==1) & (df.con==1)].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.con==1)].groupby(['vpn']).mean()
y=df_pic_pos.cor
X=pd.concat([df_pic_pos[handcrafted], df_pic_neg[handcrafted]], axis=1)
regression(X, y)

print 'Prediction of Empathy with both Muscles - Participantbased'
df_pic_pos=df[(df.val==1) & (df.con==0)].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.con==0)].groupby(['vpn']).mean()
y=np.array(df_pic_pos.key)
X=pd.concat([df_pic_pos[handcrafted], df_pic_neg[handcrafted]], axis=1)
regression(X, y)



## Classification with LogReg, RT and SVM

In [ ]:
from sklearn.model_selection import StratifiedKFold

def classify(X_train, y_train, hypercv):

    #Festlegen der mit Crossvalidierung zu tunenden Parameter
    tuned_rt_parameters = [{'max_depth':[1, 2,4,6],
                   'min_samples_leaf':[1, 2,4,6]}]

    tuned_lm_parameters = [{'C':[0.001, 0.01, 0.1, 1., 10.0,], 
                  }]

    tuned_svm_parameters = [{'C':[0.001, 0.01, 0.1, 1., 10.0],  
                  }]

    #Random-Forest Classifier mit getunten Hyperparametern        
    rf_clf = GridSearchCV(RandomForestClassifier(n_estimators=100, class_weight='balanced'), 
                                         tuned_rt_parameters, cv=hypercv) #, scoring=score

    rf_clf.fit(X_train, y_train)
    
    #Logistic Regression Classifier mit getunten Hyperparametern
    lr_clf = GridSearchCV(linear_model.LogisticRegression(class_weight='balanced'),
               tuned_lm_parameters, cv=hypercv)#, scoring=score)
    lr_clf.fit(X_train, y_train)
    
    #SVM
    svm_clf = GridSearchCV(svm.LinearSVC(class_weight='balanced'), 
                   tuned_svm_parameters, cv=hypercv)#, scoring=score)
    svm_clf.fit(X_train, y_train)
  
    return  rf_clf, lr_clf, svm_clf

In [ ]:
 def classify_cross_eval(y_true, y_pred_rf, y_pred_lr, y_pred_svm):

    #Evaluation der crossvalidierten Ergebnisse 
    print 'Crossvalidierte Ergebnisse fuer Random Forest'
    print metrics.classification_report(y_true,y_pred_rf)

    print 'Crossvalidierte Ergebnisse fuer Logistic Regression'
    print metrics.classification_report(y_true,y_pred_lr)

    print 'Crossvalidierte Ergebnisse fuer Support Vector Machine'
    print metrics.classification_report(y_true,y_pred_svm)  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import svm
from sklearn import linear_model 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


def classification(X, y, cvtt=3, hypercv=3):
    y_true_all = []
    y_pred_lr_all = []
    y_pred_rf_all = []
    y_pred_svm_all = []
    bar_svm=[]
    bar_lr=[]
    bar_rf=[]

    X=np.array(X)
    y=np.array(y)
    
    #shuffle the data 
    indices=np.arange(len(X))
    np.random.shuffle(indices)
    X=X[indices]
    y=y[indices]

    #Nested Cross-Validation    
    rs = StratifiedShuffleSplit(n_splits=cvtt, test_size=0.3)
    for train, test in rs.split(X, y):
        
        X_train=X[train]
        X_test=X[test]
        
        y_test=y[test]
        y_train=y[train]
        
        rf_clf, lr_clf, svm_clf=classify(X_train, y_train, hypercv=hypercv)
              
        evaluate(rf_clf, lr_clf, svm_clf, X_test, y_test)       
              
        y_true_all = np.append(y_true_all, y_test)
        y_pred_rf_all = np.append(y_pred_rf_all, rf_clf.predict(X_test)) 
        y_pred_lr_all = np.append(y_pred_lr_all, lr_clf.predict(X_test)) 
        y_pred_svm_all = np.append(y_pred_svm_all, svm_clf.predict(X_test)) 
        
        bar_svm=np.append(bar_svm, np.sum(svm_clf.predict(X_test)==y_test)/np.float(len(y_test)))
        bar_lr=np.append(bar_lr, np.sum(lr_clf.predict(X_test)==y_test)/np.float(len(y_test)))
        bar_rf=np.append(bar_rf, np.sum(rf_clf.predict(X_test)==y_test)/np.float(len(y_test)))

    graphics(bar_svm, bar_lr, bar_rf)
    
    cvs = StratifiedShuffleSplit(n_splits=5, test_size=0.3)
    
    train_sizes=np.linspace(.1, 1.0, 4)
    title='Learning Curve'
    try:
        plot_learning_curve(linear_model.LogisticRegression(class_weight='balanced', C=lr_clf.best_estimator_.C),
                        title, X, y, ylim=(0.4, 1.01), cv=cvs)
    except:
        print 'no learning curve for LR available'
    try:
        plot_learning_curve(svm.LinearSVC(class_weight='balanced', C=svm_clf.best_estimator_.C),
            title, X, y, ylim=(0.4, 1.01), cv=cvs)
    except:
        print 'no learning curve for SVM available'
    try:
        plot_learning_curve(RandomForestClassifier(n_estimators=100, class_weight='balanced',
                                               max_depth=rf_clf.best_estimator_.max_depth, 
                                               min_samples_leaf=rf_clf.best_estimator_.min_samples_leaf),
                        title, X, y, ylim=(0.4, 1.01), cv=cvs)
    except:
        print 'no learning curve for RT available'
    classify_cross_eval(y_true_all, y_pred_rf_all, y_pred_lr_all, y_pred_svm_all)

In [ ]:
def graphics(bar_svm, bar_lr, bar_rf):
    fig, ax = plt.subplots()
    ax.bar(1, np.mean(bar_lr), yerr=np.std(bar_lr), color='r', label='LR')#, men_means, width, color='r', yerr=men_std)
    ax.bar(2, np.mean(bar_svm), yerr=np.std(bar_svm),color='b', label='SVM')
    ax.bar(3, np.mean(bar_rf), yerr=np.std(bar_rf), color='y', label='RF')
    
    ax.set_ylabel('Accuracy')
    ax.set_title('Accuracy of the Different Classifier')
    #ax.set_xticks([1,2,3] + 1 / 2)
    #ax.set_xticklabels(('LR', 'SVM', 'RF'))st
    ax.legend()
    
    plt.show()

In [ ]:
def evaluate(rf_clf, lr_clf, svm_clf, X_test, y_test):
#Evaluation der Classifier auf den Training-Test-Splits
        
    y_pred_rf=rf_clf.predict(X_test)
    y_pred_lr=lr_clf.predict(X_test)
    y_pred_svm=svm_clf.predict(X_test)

    print("Beste Parameter fuer Random Forest auf einem der Trainingssets:")
    print(rf_clf.best_estimator_)

    print("Werte für den RF auf einem der Trainingssets:")
    print metrics.classification_report(y_test, y_pred_rf)

    print("Beste Parameter fuer Logistic Regression  auf einem der Trainingssets:")
    print(lr_clf.best_estimator_)

    print("Werte für die LR auf einem der Trainingssets:")
    print metrics.classification_report(y_test,y_pred_lr)

    print("Beste Parameter fuer SVM  auf einem der Trainingssets:")
    print(svm_clf.best_estimator_)

    print("Werte für die SVM auf einem der Trainingssets:")
    print metrics.classification_report(y_test,y_pred_svm)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

def classification_grouped(X, y, vpn, cvtt=3, hypercv=5):
      
    y_true_all = []
    y_pred_lr_all = []
    y_pred_rf_all = []
    y_pred_svm_all = []
    bar_svm=[]
    bar_lr=[]
    bar_rf=[]
    
    vpns=np.array(vpn.unique())    

    rs = ShuffleSplit(n_splits=3, test_size=0.3)
    for train, test in rs.split(vpns):
        
        X_train=np.array(X[vpn.isin(vpns[train])])
        X_test=np.array(X[vpn.isin(vpns[test])])        
        y_test =np.array(y[vpn.isin(vpns[test])])
        y_train=np.array(y[vpn.isin(vpns[train])])
        
        rf_clf, lr_clf, svm_clf=classify(X_train, y_train, hypercv=hypercv)
         
        evaluate(rf_clf, lr_clf, svm_clf, X_test, y_test)       
              
        y_true_all = np.append(y_true_all, y_test)
        y_pred_rf_all = np.append(y_pred_rf_all, rf_clf.predict(X_test)) 
        y_pred_lr_all = np.append(y_pred_lr_all, lr_clf.predict(X_test)) 
        y_pred_svm_all = np.append(y_pred_svm_all, svm_clf.predict(X_test)) 
        
        bar_svm=np.append(bar_svm, np.sum(svm_clf.predict(X_test)==y_test)/np.float(len(y_test)))
        bar_lr=np.append(bar_lr, np.sum(lr_clf.predict(X_test)==y_test)/np.float(len(y_test)))
        bar_rf=np.append(bar_rf, np.sum(rf_clf.predict(X_test)==y_test)/np.float(len(y_test)))
    
    #eigentlich muesste man hier die gemittelten best_estimator nehmen

    graphics(bar_svm, bar_lr, bar_rf)
    
    cvs = StratifiedShuffleSplit(n_splits=5, test_size=0.3)
                              
    train_sizes=np.linspace(.1, 1.0, 5)
    title='Learning Curve'
    
    try:
        plot_learning_curve(linear_model.LogisticRegression(class_weight='balanced', C=lr_clf.best_estimator_.C),
                        title, X, y, (0.4, 1.01), cv=cvs, train_sizes=train_sizes) 
    
    except:
         print 'no learning curve for LR available'
    try:
        plot_learning_curve(RandomForestClassifier(n_estimators=100, class_weight='balanced',
                                               max_depth=rf_clf.best_estimator_.max_depth, 
                                               min_samples_leaf=rf_clf.best_estimator_.min_samples_leaf),
                        title, X, y, (0.4, 1.01), cv=cvs)
    except:
        print 'no learning curve for RF available'
    classify_cross_eval(y_true_all, y_pred_rf_all, y_pred_lr_all, y_pred_svm_all)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.show()

## Classification (Valence, Condition Correctness)

In [ ]:
df=df.dropna(subset=zyg_s).reset_index(drop=True)
df=df.dropna(subset=cor_s).reset_index(drop=True)

try:
    df.loc[(df['val']=='pos'), 'val']=1
    df.loc[(df['val']=='neg'), 'val']=0
except:
    print'valence already recoded'
    
try:
    df.loc[(df['con']==2), 'con']=0
except:
    print'condition already recoded'
       
df['con']=pd.to_numeric(df.con)
df['val']=pd.to_numeric(df.val)

### Valence of a Picture

In [ ]:
print 'Prediction of valence with both Muscles'
X=pd.concat([df[zyg_s], df[cor_s]], axis=1)
y=df['val']
vpns=df['vpn']
classification_grouped(X, y, vpns)

In [ ]:
print 'Prediction of valence with handcrafted features'
X=df[handcrafted]
y=df['val']
vpns=df['vpn']
classification_grouped(X,y, vpns)

#### picture-based

In [ ]:
print 'Prediction of valence with both Muscles (pic-based)'
df_pic=df.groupby(['pic']).mean().reset_index()
x1=df_pic[zyg_s]
x2=df_pic[cor_s]
y=df_pic['val']
classification(pd.DataFrame(pd.concat([x1, x2], axis=1)), y)

In [ ]:
print 'Handcrafted'
print 'Prediction of valence with both Muscles (pic-based)'
df_pic=df.groupby(['pic']).mean().reset_index(drop=True)
y=df_pic['val']
classification(df_pic[handcrafted], y)

### Condition of a Picture

In [ ]:
print 'Prediction of condition with both Muscles'
X=pd.concat([df[zyg_s], df[cor_s]], axis=1)
y=df['con']
vpns=df['vpn']
classification_grouped(X,y, vpns)

In [ ]:
print 'Prediction of condition with both Muscles'
X=df[handcrafted]
y=df['con']
vpns=df['vpn']
classification_grouped(X,y, vpns)

In [ ]:
print 'PICTUREBASED'
print 'Prediction of condition with both Muscles (pic-based)'
cog_df=df[df.con==1].groupby(['pic']).mean().reset_index()
emo_df=df[df.con==0].groupby(['pic']).mean().reset_index()
Xz=pd.concat([cog_df[zyg_s], emo_df[zyg_s]], axis=0)
Xc=pd.concat([cog_df[cor_s], emo_df[cor_s]], axis=0)
X=pd.concat([Xz, Xc], axis=1).reset_index(drop=True)
y=pd.concat([cog_df['con'], emo_df['con']], axis=0).reset_index(drop=True)
vpns=pd.concat([cog_df['pic'], emo_df['pic']], axis=0).reset_index(drop=True)
classification_grouped(X,y, vpns)

In [ ]:
print 'Handcrafted'
print 'Prediction of condition with both Muscles (pic-based)'
cog_df=pd.DataFrame(df[df.con==1]).groupby(['pic']).mean().reset_index()
emo_df=pd.DataFrame(df[df.con==0]).groupby(['pic']).mean().reset_index()
X=pd.concat([cog_df[handcrafted], emo_df[handcrafted]], axis=0)
y=pd.concat([cog_df['con'], emo_df['con']], axis=0)
vpns=pd.concat([cog_df['pic'], emo_df['pic']], axis=0)
classification_grouped(X,y, vpns)


### Correct Answers

In [ ]:
#print 'Prozentsatz korrekt beantworte Items' 
#print np.divide(sum(cog_df['cor']==1),float(sum(cog_df['cor']==0)+sum(cog_df['cor']==1)))

print 'Prediction of correct Anwsers by muscle activity'
cog_df=df[df.con==1].reset_index(drop=True)
X=pd.concat([cog_df[zyg_s], cog_df[cor_s]], axis=1)
y=cog_df['cor']
vpns=cog_df['vpn']
classification_grouped(X,y, vpns)

In [ ]:
print 'Handcrafted'
print 'Prediction of correct Anwsers by muscle activity'
cog_df=df[df.con==1].reset_index(drop=True)
X=cog_df[handcrafted]
y=cog_df['cor']
vpns=cog_df['vpn']
classification_grouped(X,y, vpns)

## T-Test of Prediction vs. Baseline-Prediction

In [ ]:
print 'Prediction of Empathy with both Muscles - PICTUREBASED'
df_pic=df.loc[df.con==0].groupby(['pic']).mean()
y=df_pic.key
X=pd.concat([df_pic[zyg_s], df_pic[cor_s]], axis=1)

error_lin_all=[]
error_svm_all=[]
error_mean_all=[]

for i in np.arange(1000):
    error_lin, error_svm, error_mean=regression(X, y)
    error_lin_all = np.append(error_lin_all, error_lin)
    error_svm_all = np.append(error_svm_all, error_svm)
    error_mean_all = np.append(error_mean_all, error_mean)
    
import scipy
print scipy.stats.ttest_rel(error_lin_all, error_mean_all)
print scipy.stats.ttest_rel(error_svm_all, error_mean_all)
print np.nanmean(error_svm_all)
print np.nanmean(error_lin_all)
print np.nanmean(error_mean_all)

print np.var(error_lin_all)
print np.var(error_svm_all)
print scipy.stats.wilcoxon(error_lin_all, error_mean_all, zero_method='wilcox', correction=False)
print scipy.stats.wilcoxon(error_svm_all, error_mean_all, zero_method='wilcox', correction=False)

In [ ]:
print np.sum(error_lin_all>error_mean_all)
print np.sum(error_svm_all>error_mean_all)
print np.std(error_lin_all)
print np.std(error_svm_all)
print np.std(error_mean_all)

In [ ]:
print 'Prediction of AQ with both Muscles (only stimulus-time) - PARTICIPANTBASED'
df_pic_pos=df[(df.val==1) & (df.AQsum.notnull())].groupby(['vpn']).mean()
df_pic_neg=df[(df.val==0) & (df.AQsum.notnull())].groupby(['vpn']).mean()
y=df_pic_pos.AQsum
X=pd.concat([df_pic_pos[zyg_s], df_pic_pos[cor_s], df_pic_neg[zyg_s], df_pic_neg[cor_s]], axis=1)

error_lin_all=[]
error_svm_all=[]
error_mean_all=[]

for i in np.arange(1000):
    error_lin, error_svm, error_mean=regression(X, y)
    error_lin_all = np.append(error_lin_all, error_lin)
    error_svm_all = np.append(error_svm_all, error_svm)
    error_mean_all = np.append(error_mean_all, error_mean)
    
import scipy
print scipy.stats.ttest_rel(error_lin_all, error_mean_all)
print scipy.stats.ttest_rel(error_svm_all, error_mean_all)
print np.nanmean(error_svm_all)
print np.nanmean(error_lin_all)
print np.nanmean(error_mean_all)
print np.var(error_lin_all)
print np.var(error_svm_all)
print scipy.stats.wilcoxon(error_lin_all, error_mean_all, zero_method='wilcox', correction=False)
print scipy.stats.wilcoxon(error_svm_all, error_mean_all, zero_method='wilcox', correction=False)

In [ ]:
print np.std(error_lin_all)
print np.std(error_svm_all)
print np.std(error_mean_all)
np.sum(error_svm_all>error_mean_all)

In [ ]:
print 'Prediction of condition with Mimicry'
X=np.array(df[zyg_s])-np.array(df[cor_s])
y=np.array(df['con'])
vpns=df['vpn']
classification_grouped(X,y, vpns)

In [ ]:
print 'Prediction of correct Anwsers by Mimicry'
cog_df=df[df.con==1].reset_index(drop=True)
X=np.array(cog_df[zyg_s])-np.array(cog_df[cor_s])
y=np.array(cog_df['cor'])
vpns=cog_df['vpn']
classification_grouped(X,y, vpns)

In [ ]:
import DTW

from sklearn.model_selection import train_test_split
import pandas as pd

def d_DTW(x, x2, dist):
    t1, t2 = len(x), len(x2)
    
    if x == [] and x2 == []:
        return 0.0
    elif (x == []) or (x2 == []):
        return np.infty
    
    dp = np.empty((t1+1, t2+1))    
    dp[0, 0] = 0
    
    for i in xrange(1, t1+1):
        dp[i, 0] = np.infty
    
    for j in xrange(1, t2+1):
        dp[0, j] = np.infty
        
    for k in xrange(1, t1+t2+1):
        for i in xrange(max(1, k - t2), min(k, t1+1)):
            j = k - i      
            dp[i, j] = dist(x[i-1], x2[j-1]) + min(dp[i-1, j-1], dp[i, j-1], dp[i-1, j])
            # print i, j, dp[i, j]
    
    return dp[t1, t2]


def d1(x, x2):
    return 0 if x != x2 else 1

def d2(x, x2):
    return (x - x2)**2

def d3(x, x2):
    return abs(x - x2)


def build_dtw_gram_matrix(xs, x2s, k):
    """
    xs: collection of sequences (vectors of possibly varying length)
    x2s: the same, needed for prediction
    k: a kernel function that maps two sequences of possibly different length to a real
    The function returns the Gram matrix with respect to k of the data xs.
    """
    t1, t2 = len(xs), len(x2s)
    K = np.empty((t1, t2))
    
    for i in xrange(t1):
        for j in xrange(i, t2):
            K[i, j] = k(xs[i], x2s[j])
            if i < t2 and j < t1:
                K[j, i] = K[i, j]
        
    return K
    
    
def L2_reg(w, lbda):
    return 0.5 * lbda * (np.dot(w.T, w)), lbda*w

def hinge_loss(h, y):
    n = len(h)
    l = np.maximum(0, np.ones(n) - y*h)
    g = -y * (h > 0)
    return l, g


def learn_reg_kernel_ERM(X, y, lbda, k, loss=hinge_loss, reg=L2_reg, max_iter=200, tol=0.001, eta=1., verbose=False):
    """Kernel Linear Regression (default: kernelized L_2 SVM)
    X -- data, each row = instance
    y -- vector of labels, n_rows(X) == y.shape[0]
    lbda -- regularization coefficient lambda
    k -- the kernel function
    loss -- loss function, returns vector of losses (for each instance) AND the gradient
    reg -- regularization function, returns reg-loss and gradient
    max_iter -- max. number of iterations of gradient descent
    tol -- stop if norm(gradient) < tol
    eta -- learning rate
    """
    num_features = X.shape[1]
    
    g_old = None
    
    K = build_dtw_gram_matrix(X, X, k) #
    w = np.random.randn(K.shape[0]) #w = np.random.randn(num_features)
    
    for _ in xrange(max_iter):
        h = np.dot(K, w) # h = np.dot(X, w)
        l,lg = loss(h, y)
        
        if verbose:
            print 'training loss: {}'.format(np.mean(l))
            
        r,rg = reg(w, lbda)
        g = lg + rg 
        
        if g_old is not None:
            eta = eta*(np.dot(g_old.T,K).dot(g_old))/(np.dot((g_old - g).T, K).dot(g_old))
            # eta = eta*(np.dot(g_old.T,g_old))/(np.dot((g_old - g).T, g_old))
            
        w = w - eta*g
        if (np.linalg.norm(eta*g)<tol):
            break
        g_old = g
        
    return w, K


def predict(alpha, X, X_train, k):
    K = build_dtw_gram_matrix(X_train, X, k)
    y_pred = np.dot(K, alpha)
    y_pred[y_pred >= 0] = 1
    y_pred[y_pred < 0] = -1
    
    return y_pred


from sklearn.learning_curve import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1,
                        train_sizes=10, # list of floats that describe ratio of test data sets tried
                        # OR an int = # how many trials
                        scoring=None):

    if type(train_sizes) == int:
        train_sizes=np.linspace(.1, 1.0, train_sizes)

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
 
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    if cv is not None:
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    if cv is not None:
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


from sklearn.base import BaseEstimator

class KernelEstimator(BaseEstimator):
    
    def __init__(self, k, lbda):
        self.k = k
        self.lbda = lbda
        
    def fit(self, X, y):
        self._X_train = X
        self._alpha, _ = learn_reg_kernel_ERM(X, y, lbda=self.lbda, k=self.k, max_iter=20000, eta=1, tol=1e-3)
        return self
    
    def predict(self, X):
        return predict(self._alpha, self._X_train, X, self.k)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y == y_pred)
        

def dtwkernel(X,y):
    k1_hyp, k2_hyp, k3_hyp = [lambda lmbd: (lambda x, x2: np.exp(-lmbd * DTW.d_DTW(x, x2, d))) for d in [DTW.d1, DTW.d2, DTW.d3]]
    k1 = k1_hyp(2.0)
    k2 = k2_hyp(2.0)
    k3 = k3_hyp(2.0)

    X=np.array(X)
    y=np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    alpha, K = DTW.learn_reg_kernel_ERM(X_train, y_train, lbda=1, k=k2, max_iter=500, eta=1, tol=1e-3, verbose=True)
    
    y_pred = DTW.predict(alpha, X_train, X_train, k2)
    print "Training Accuracy: {}".format(np.mean(y_train == y_pred))
    print "Test Accuracy: {}".format(np.mean(y_test == DTW.predict(alpha, X_train, X_test, k2)))
    
    estimator = DTW.KernelEstimator(k2, 2.0)
    DTW.estimator.fit(X_train, y_train)
    print "Accuracy {}".format(estimator.score(X_train, y_train))
    

#PREDICTION OF THE VALENCE OF A PIC (averaged above participants)
print 'Prediction of valence with both Muscles'
#df.loc[(df['val']=='pos'), 'val']=1
#df.loc[(df['val']=='neg'), 'val']=0

df['val']=pd.to_numeric(df.val)
df_nn=df.dropna(subset=zyg_s)
df_pic=df.groupby(['pic']).mean().reset_index()
x=pd.DataFrame(np.array(df_nn[cor_s])-np.array(df_nn[zyg_s]))
y=df_nn['val']

#print y.dropna(subset=x)
dtwkernel(x,y)

print 'Prediction of correct Anwsers by muscle activity'
#cog_df=(df.loc[df['con']==1])   
#cog_df = cog_df.reset_index(drop=True)

#y=np.array(cog_df.cor)
#x=cog_df[zyg_s]
#dtwkernel(x,y)

#x=df_pic[cor_s]
#dtwkernel(X,y)


# estimator = DTW.KernelEstimator(k2, 2.0)
#DTW.estimator.fit(X_train, y_train)
#print "Accuracy {}".format(estimator.score(X_train, y_train))

#DTW.plot_learning_curve(KernelEstimator(k2, 2.0), 'Euclidean distance DTW, lambda = 1.0', X_train, y_train, cv=None, scoring="accuracy", train_sizes=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0])